In [ ]:
!pip -q install langchain_google_genai
!pip -q install python-dotenv

In [ ]:
from langchain_google_genai import ChatGoogleGenAI
from dotenv import load_dotenv

load_dotenv()
chat_google = ChatGoogleGenAI(model='gemnini-1.5-pro')
result = chat_google.invoke("what is the capital of india?")
print(result.content)


In [ ]:
!pip -q install langchain_huggingface

In [3]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import warnings
warnings.filterwarnings('ignore')
import os
hf_token = os.getenv("HUGGINGFACEHUB_ACCESS_TOKEN")

llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-20b",
    task="text-generation",
    huggingfacehub_api_token=hf_token
)

model = ChatHuggingFace(llm=llm)
response = model.invoke("what is the capital of india.")
print(response.content)


The capital of India is **New Delhi**.


In [ ]:

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-20b",
    task="text-generation",
    huggingfacehub_api_token=hf_token
)

model = ChatHuggingFace(llm=llm)

memory = ConversationBufferWindowMemory(k=3, memory_key="history",return_messages=True)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Order Tracking Assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = LLMChain(
    llm=model,
    prompt=prompt,
    memory=memory
)

while True:
    user_input = input("prompt: ")
    print(f'Prompt: {user_input}')
    if user_input.lower() == 'exit':
        print('Response : GoodBye....')
        break

    response = chain.invoke({"input": user_input})
    print(f"Response: {response['text']}\n")


**HuggingFacePipeline**

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="openai/gpt-oss-20b",
    task="text-generation",
    pipelin_kwargs=dict(
        temperatur=0.5,
        max_new_tokens=100
    )

)
model = ChatHuggingFace(llm=llm)

result = model.invoke("what is the capital of india.")
print(result.content)

3. Embedding Models

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="openai/gpt-oss-20b",
    task="text-generation",
    pipelin_kwargs=dict(
        temperatur=0.5,
        max_new_tokens=100
    )

)
model = ChatHuggingFace(llm=llm)

result = model.invoke("what is the capital of india.")
print(result.content)

In [ ]:
!pip -q install langchain_openai

In [ ]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=32)

documents = [
    "Delhi is the capital of India",
    "Mumbai is the capital of Maharastra",
    "Kolkata is the capital of West Bengal",
    "Hyderabad is the capital of Telangana"
]
result = embedding.embed_query("Delhi is the capital of India")
print(str(result))


result = embedding.embed_documents(documents)
print(str(result))


**all-MiniLM-L6-v2**
1. This is a sentence-transformers model:
* It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.

2. Usage (Sentence-Transformers)
* Using this model becomes easy when you have sentence-transformers installed:

```
pip install -U sentence-transformers
```







In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', )
text = 'This is a test document.'
result = embedding.embed_query(text)
print(str(result))


documents = [
    "Delhi is the capital of India",
    "Mumbai is the capital of Maharastra",
    "Kolkata is the capital of West Bengal",
    "Hyderabad is the capital of Telangana"
]

result = embedding.embed_documents(documents)
print(str(result))

**Semantic Search Using the cosine similary and Embeddings**

In [ ]:
!pip -q install langchain_openai

In [ ]:

from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=300)

documents = [
    "Virat Kohli is an Indian cricketer known for his aggressive batting and leadership.",
    "MS Dhoni is a former Indian captain famous for his calm demeanor and finishing skills.",
    "Sachin Tendulkar, also known as the 'God of Cricket', holds many batting records.",
    "Rohit Sharma is known for his elegant batting and record-breaking double centuries.",
    "Jasprit Bumrah is an Indian fast bowler known for his unorthodox action and yorkers."
]

query = 'tell me about bumrah'

doc_embeddings = embedding.embed_documents(documents)
query_embedding = embedding.embed_query(query)

scores = cosine_similarity([query_embedding], doc_embeddings)[0]

index, score = sorted(list(enumerate(scores)),key=lambda x:x[1])[-1]

print(query)
print(documents[index])
print("similarity score is:", score)

<h2>Prompts</h2>

1.   Text based prompt
2.   Multimodel prompt - text, audio, video, images



1.  The below code uses the native memory for maintaining the conversational history.
2.  For the role in the memory i have used the SystemMessage, HumanMessage, AIMessage which differentiates the roles in history.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

model = HuggingFaceEndpoint(repo_id='openai/gpt-oss-20b', task='text-generation', huggingfacehub_api_token=hf_token)
model = ChatHuggingFace(llm=model)

history = [SystemMessage(content='You are a helpful assistant')]

while True:
  prompt = input("Prompt: ")
  if prompt.lower() == 'exit':
    break
  history.append(HumanMessage(content=prompt))
  response = model.invoke(history)
  print(f'AI: {response.content:}')
  history.append(AIMessage(content=response.content))


* The below code download the model locally from Huggingface and uses it for text generation.

In [ ]:
!pip -q install -U langchain_huggingface

In [5]:
!pip -q install -U transformers


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline


llm = HuggingFacePipeline.from_model_id(
    model_id="LiquidAI/LFM2-1.2B",
    task="text-generation",
    pipeline_kwargs=dict(
        temperature=0.5,
        max_new_tokens=100
    )
)

model = ChatHuggingFace(llm=llm)

result = model.invoke("What is the capital of India?")
print(result.content)


Device set to use cpu


<|startoftext|><|im_start|>user
What is the capital of India?<|im_end|>
<|im_start|>assistant
The capital of India is New Delhi. It has been serving as the capital since 1876, although the territory now known as New Delhi was not officially established as the capital until 1911 when the British government moved the capital from Calcutta (now Kolkata) to Delhi. New Delhi is the political, administrative, and cultural hub of India, housing numerous important government buildings, embassies, and historical monuments.
